In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs_min = "20250620"
#day_obs_min = "yesterday"
day_obs_max = "20250922"
#day_obs_max = "2024-12-10"
#instrument = "latiss"  
#instrument = "lsstcomcam"
instrument = "lsstcam"

# ConsDB Visit Metadata from {{ params.day_obs_min }} to {{ params.day_obs_max }}

In [ ]:
import os
import copy
from math import floor
import numpy as np
import healpy as hp
import matplotlib.pylab as plt
from cycler import cycler
import colorcet as cc

import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u
import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

import logging

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

In [ ]:
import os
if os.getenv("EXTERNAL_INSTANCE_URL") is not None:
    print("updating rubin_nights")
    !pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git  --no-deps  > /dev/null 2>&1

from rubin_nights import connections
from rubin_nights.observatory_status import get_dome_open_close
from rubin_nights.augment_visits import augment_visits
import rubin_nights.rubin_scheduler_addons as rn_sch
import rubin_nights.dayobs_utils as rn_dayobs

In [ ]:
if isinstance(day_obs_min, str):
    if day_obs_min.lower() == "today":
        day_obs_min = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
    elif day_obs_min.lower() == "yesterday":
        day_obs_min = rn_dayobs.day_obs_str_to_int(rn_dayobs.yesterday_day_obs())
if isinstance(day_obs_max, str):
    if day_obs_max.lower() == "today":
        day_obs_max = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
    elif day_obs_max.lower() == "yesterday":
        day_obs_max = rn_dayobs.day_obs_str_to_int(rn_dayobs.yesterday_day_obs())

try:
    t_start = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_min)}T12:00:00", format='isot', scale='utc')
except ValueError:
    print(f"Is day_obs_min the right format? {day_obs_min} should be YYYYMMDD")
    t_start = None
try:
    t_end = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format='isot', scale='utc') + TimeDelta(1, format='jd')
except ValueError:
    print(f"Is day_obs_max the right format? {day_obs_max} should be YYYYMMDD")
    t_start = None

if t_start is None or t_end is None:
    print("Did not get valid inputs for time period.")

In [ ]:
day_min = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_min)}T12:00:00", format='isot', scale='utc')
day_max = Time(f"{rn_dayobs.day_obs_int_to_str(day_obs_max)}T12:00:00", format='isot', scale='utc')
one_day = TimeDelta(1, format='jd')
days = day_min + one_day * np.arange(0, (day_max - day_min).jd + 1)
day_obs_list = [d.iso[0:10] for d in days]

## Accessing visits from ConsDB (USDF)

In [ ]:
if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

# Hack for Lynne - but you can use your own token (outside of RSP)
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    tokenfile = '/Users/lynnej/.lsst/usdf_rsp'
    site = 'usdf'

else:
    tokenfile = None
    site = None

endpoints = connections.get_clients(tokenfile, site)
consdb = endpoints['consdb']

In [ ]:
visits = consdb.get_visits(instrument, day_min, day_max + one_day)

if len(visits) > 0:
    display(Markdown(f"Retrieved {len(visits)} visits from consdb from {day_min.iso} to {(day_max + one_day).iso}"))
if len(visits) == 0:
    display(Markdown(f"No visits for {instrument} between {day_obs_min} to {day_obs_max} retrieved from consdb"))

In [ ]:
if len(visits) > 0:
    # calculate slew times 
    wait_before_slew = 1.45
    settle = 2.0
    visits, slewing = rn_sch.add_model_slew_times(visits, endpoints['efd'], model_settle=wait_before_slew + settle, 
                                                  dome_crawl=False, slew_while_changing_filter=False)
    
    max_scatter = 6
    valid_overhead = np.min([np.where(np.isnan(visits.slew_model.values), 0, visits.slew_model.values) + max_scatter, visits.visit_gap.values], axis=0)
    visits['overhead'] = valid_overhead
    
    visits['fault'] = visits.visit_gap - visits.overhead

In [ ]:
# Get dome open/close times
dome_open = get_dome_open_close(day_min, day_max + one_day, endpoints['efd'])

In [ ]:
## Almanac ## 
display(Markdown(f"## Daily information for {day_obs_min} to {day_obs_max}"))
site = Site('LSST')
almanac = Almanac()
alm = {}
if len(visits) > 0:
    for day_obs in day_obs_list:
        dayobsint = int(day_obs.replace('-', ''))
        # almanac events
        alm[day_obs] = {}
        night_events = almanac.get_sunset_info(evening_date=day_obs, longitude=site.longitude_rad)
        horizon_sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
        horizon_sunrise = Time(night_events['sunrise'], format='mjd', scale='utc')
        sunset = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
        sunrise = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
        close = horizon_sunrise - TimeDelta(2*60*60, format='sec')
        # dome open/close events
        dd = dome_open.query("day_obs == @dayobsint")
        if len(dd) == 0:
            dome_open_hours = 0
            dome_start = None
            dome_end = None
            start_time = sunset
            end_time = sunset
        else:
            dome_open_hours = dd.open_hours.sum()
            dome_start = Time(dd.open_time.min())
            dome_end = Time(dd.close_time.max())
            start_time = np.max([sunset, dome_start])
            end_time = np.min([close, dome_end])
        night_hours = (close.mjd - sunset.mjd)*24
        night_length = (sunrise.mjd - sunset.mjd) * 24
        open_hours = (end_time.mjd - start_time.mjd) * 24

        if dome_start is not None:
            alm[day_obs]['dome open'] = dome_start.iso
        else:
            alm[day_obs]['dome open'] = np.nan
        alm[day_obs]['horizon sunset'] = horizon_sunset.iso
        alm[day_obs]['sunset'] = sunset.iso
        alm[day_obs]['admin end'] = close.iso
        if dome_end is not None:
            alm[day_obs]['dome close'] = dome_end.iso
        else:
            alm[day_obs]['dome close'] = np.nan
        alm[day_obs]['sunrise'] = sunrise.iso
        
        if np.isnan(night_events['moonrise']):
            alm[day_obs]['moon rise'] = np.nan
        else:
            alm[day_obs]['moon rise'] = Time(night_events['moonrise'], format='mjd', scale='utc').iso
        if np.isnan(night_events['moonset']):
            alm[day_obs]['moon set'] = np.nan
        else:
            alm[day_obs]['moon set'] = Time(night_events['moonset'], format='mjd', scale='utc').iso
        moon_phase = almanac.get_sun_moon_positions(sunset.mjd)['moon_phase']
        alm[day_obs]['moon phase'] = moon_phase.round(2)
        

        alm[day_obs]['dome hours'] = round(dome_open_hours, 3)
        alm[day_obs]['night hours'] =  night_hours.round(3) 
        alm[day_obs]['avail hours'] = round(open_hours, 3)

        # Visits and visit hours
        vv = visits.query('day_obs == @dayobsint')
        alm[day_obs]['programs'] = vv.science_program.unique()
        alm[day_obs]['nvisits dayobs'] = len(vv)
        
        vv = visits.query("day_obs == @dayobsint and obs_start_mjd >= @start_time.mjd and obs_end_mjd <= @end_time.mjd")

        if len(vv) == 0:
            alm[day_obs]['fault'] = round(open_hours, 3)
            cols_to_zero = ['night exptime', 'night slews', 'night frac', 'nvisits night',
                            'start to night visits', 'night visits to close']
            for c in cols_to_zero:
                alm[day_obs][c] = 0

        else:
            alm[day_obs]['nvisits night'] = len(vv)    
            alm[day_obs]['start to night visits'] = ((vv.exp_midpt_mjd.min() - start_time.mjd) * 24).round(3)
            alm[day_obs]['night visits to close'] = ((end_time.mjd - vv.exp_midpt_mjd.max()) * 24).round(3)                    
            alm[day_obs]['night exptime'] = vv.dark_time.sum() / 60 / 60
            alm[day_obs]['night slews'] = vv.overhead.values[1:].sum() / 60 / 60 
            alm[day_obs]['fault'] = vv.fault.values[1:].sum() / 60 / 60
            
        if open_hours == 0:
            onsky_frac = 0
        else:
            onsky_frac =  (vv.exp_time.sum() / (open_hours * 60 * 60)).round(3)
            
        alm[day_obs]['night frac'] = onsky_frac
        vv = visits.query("day_obs == @dayobsint and science_program == 'BLOCK-365'")
        alm[day_obs]['sci exptime'] = vv.dark_time.sum() / 60 / 60
        alm[day_obs]['sci slews'] = vv.overhead.sum() / 60 / 60
        if open_hours == 0:
            sci_frac = 0
        else:
            sci_frac =  (vv.exp_time.sum() / (open_hours * 60 * 60)).round(3)
        alm[day_obs]['sci frac'] = sci_frac

    alm = pd.DataFrame(alm).T
    cols = ['dome hours', 'night hours', 'avail hours', 'night frac', 'sci frac']
    for c in cols:
        alm[c] = alm[c].astype(float)

    print("All time durations are in hours")
    print("Nvisits night are total number of visits while the dome is open and the night is in progress.")    
    print("Sunset or rise without qualifier is -12 degree sunset or sunrise")
    print("Night hours is -12 degree sunset to administrative close at 2 hours for 0-deg sunrise.")
    print("Available hours is the period of night hours where the dome is also open.")
    print("sci frac is science exposure time / available hours.")
    with option_context('display.max_colwidth', None):
        display(HTML(alm.to_html()))

In [ ]:
if len(alm) < 10:
    xlen = 12
elif len(alm) > 60:
    xlen = int(len(alm) / 4.7)
else:
    xlen = int(len(alm) / 2.5)
    
fig, ax1 = plt.subplots(figsize=(xlen, 5))
x = np.arange(0, len(alm.index.values))

ax1.plot(x, alm['night hours'], color='darkblue', linestyle=':', label="Night hours")
# dome closed hours = night hours - available hours
ax1.bar(x, alm['night hours'] - alm['avail hours'], bottom=alm['avail hours'], width=1, color='lightgray', alpha=0.8, label="Hours dome closed")
# In practice, this is identical to the effect of the bar above .. 
#ax1.bar(x, alm['night hours'], color='skyblue', alpha=0.9, width=1, label="Night time hours")

# Add unaccounted for hour space taker, as well as border around each night
ax1.bar(x, alm['avail hours'], width=1, color="white", edgecolor="black", label="Unaccounted Hours")
ax1.bar(x, alm['avail hours'], width=1, color="none", edgecolor='black', zorder=10,) #label="Available hours", zorder=10)

# Now add actual time
night_ends = alm['start to night visits'] + alm['night visits to close']
ax1.bar(x, height=night_ends, color='lightslategray', alpha=0.75, bottom = alm['night exptime'] + alm['night slews'] + alm['fault'], width=1, label="Hours Night Ends")
fault_idle = alm['fault'] 
ax1.bar(x, height=fault_idle, color='lightslategray', bottom = alm['night exptime'] + alm['night slews'], width=1, label="Hours Fault/Idle")

ax1.bar(x, height=alm['night slews'] - alm['sci slews'], bottom=alm['night exptime'] + alm['sci slews'],  color='#e28743', width=1, label="Hours Other Slew")
ax1.bar(x, alm['night exptime'] - alm['sci exptime'], bottom = alm['sci exptime'] + alm['sci slews'], color='#ffe69a', width=1, label="Hours Other Exptime")

ax1.bar(x, height=alm['sci slews'], bottom=alm['sci exptime'], color='#1e81b0', width=1, label="Hours B365 Slew")
ax1.bar(x, height=alm['sci exptime'], width=1, color='#6dcce0', alpha=1, label="Hours B365 Exptime")

ax1.legend(loc=(-0.24, 0.15), ncols=1)

ax2 = ax1.twinx()
ax2.plot(x, alm['sci frac'], 'k*', linestyle=':', linewidth=1, markerfacecolor="none", markersize=8, label="BLOCK-365\nefficiency")
ax2.legend(loc=(1.04, 0.15), ncols=1)

ax1.set_xlim(x.min()-1, x.max()+1)
ax1.set_xticks(x)
ax1.set_xticklabels(labels=alm.index.values, rotation=90)
ax1.set_ylim(0, 12)
ax1.set_ylabel("Hours", fontsize='x-large', loc='top')
ax2.set_ylim(0, 0.75)
_ = ax2.set_ylabel("Open Shutter Fraction", fontsize='large', loc='top')
#plt.savefig("onsky_hours.png", bbox_inches='tight')

In [ ]:
# fig, ax = plt.subplots(figsize=(20, 5))
# x = np.arange(0, len(alm.index.values))
# y = (alm["sci slews"] + alm["sci exptime"]) / alm["avail hours"]
# ax.plot(x, y, 'k.')
# ax.fill_betweenx(y=np.arange(0, 1, 0.1), x1=11, x2=33, color='pink', alpha=0.3)
# ax.grid(alpha=0.3)
# ax.set_xlim(x.min()-1, x.max()+1)
# ax.set_xticks(x)
# _ = ax.set_xticklabels(labels=alm.index.values, rotation=90)
# _ = ax.set_ylabel("SV Operation Hours / Available Hours")

# yy = y.values[11:33].astype(float)
# print(f"[pink shaded region] - mean availbility (discounting closed dome time): {np.nanmean(yy):.2f}")

In [ ]:
c = None
if len(visits) > 0:
    groupcols = ['science_program', 'img_type', 'target_name', 'observation_reason', 'day_obs', 'visit_id'] 
    c = visits[groupcols].groupby(['science_program', 'img_type'], dropna=False).agg({'science_program' : ['first'],
                                                                        'target_name' : ['unique'], 
                                                                        'observation_reason' : ['unique'],
                                                                        'day_obs' : ['nunique'],
                                                                        'visit_id' : ['first', 'last', 'count']})

In [ ]:
# This might work .. to help translate test block numbers above into more meaningful programs
testcase_base_url = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/v2/testCase/"
jiraticket_base_url = "https://rubinobs.atlassian.net/browse/"

if len(visits) > 0:
    jira_urls = {}
    for science_program in c.sort_values(by=('visit_id', 'first'))[('science_program', 'first')].values: # visits.science_program.unique():
        if science_program is not None and science_program.startswith("BLOCK-T"):
            jira_urls[science_program] = testcase_base_url + science_program
        elif science_program is not None and science_program.startswith("BLOCK-"):
            jira_urls[science_program] = jiraticket_base_url + science_program
        else:
            jira_urls[science_program] = ''
            #display(Markdown(f"[{science_program}]({jira_url}) - {test_name} ({len(visits.query('science_program == @science_program'))} visits)"))

In [ ]:
if len(visits) > 0:
    jira_url_col = np.array(["X" *  (max([len(v) for v in jira_urls.values()])+100)] * len(c))
    for i, (ri, row) in enumerate(c.iterrows()):
        sp = row[('science_program', 'first')]
        if sp in jira_urls.keys():
            jira_url_col[i] = f'<a href="{jira_urls[sp]}" target="_blank" rel="noreferrer noopener">{sp}</a>'
        else:
            jira_url_col[i] = ''
    c['JIRA'] = jira_url_col

In [ ]:
if len(visits) > 0:
    display(Markdown(f"ConsDB Visits"))
    with option_context('display.max_colwidth', None):
        cols = [('JIRA', ''), ('target_name', 'unique'), ('observation_reason', 'unique'), ('day_obs', 'nunique'),
                ('visit_id', 'first'), ('visit_id', 'last'), ('visit_id', 'count')]
        display(HTML(c[cols].sort_values(by=('visit_id', 'first')).to_html(escape=False)))

## Visits vs Time properties

This is a visits vs time example from schedview, using better Rubin color mapping for the various filters.  Due to memory and size, this runs on the last three nights only. (download notebook and run yourself for more). 

In [ ]:
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    os.environ["SCHEDVIEW_DATASOURCE_URL"] = endpoints['api_base']
    os.environ["SCHEDVIEW_EFD_NAME"] = 'usdf_efd'

import warnings
import bokeh.io

bokeh.io.output_notebook()

logging.getLogger('rubin_sim').setLevel(logging.ERROR)
logging.getLogger('schedview').setLevel(logging.ERROR)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import schedview.plot

from rubin_sim.maf import ObservationStartTimestampStacker

print("Shows only up to the last 3 nights with any data of the timespan")

# Hack the tooltip - Eric will fix this a different way soon.
def visits_tooltips() -> list:
    deg = "\u00b0"
    tooltips = [
        (
            "Start time",
            "@start_timestamp{%F %T} UTC"
        ),
        ("VisitId", "@visit_id"),
        ("Band", "@band"),
        (
            "Field coordinates",
            "RA=@s_ra" + deg + ", Decl=@s_dec" + deg + ", Az=@azimuth" + deg + ", Alt=@altitude" + deg,
        ),
        ("Target name", "@target_name"),
        ("Observation reason", "@observation_reason"),
        ("Science program", "@science_program"),
        ("Scheduler note", "@scheduler_note"),
    ]
    return tooltips

schedview.plot.visits.visits_tooltips = visits_tooltips

if len(visits):
    timestamp_stacker = ObservationStartTimestampStacker(mjd_col="obs_start_mjd",)
    dayobs_cutoff = int((day_max - TimeDelta(3, format='jd')).iso[0:10].replace('-', ''))
    vv = visits.query("day_obs > @dayobs_cutoff and (img_type == 'science' or img_type == 'acq')").copy()
    # extra_lookback = 0
    # while len(vv) == 0 and extra_lookback < 5:
    #     extra_lookback += 1
    #     dayobs_cutoff = int((day_max - TimeDelta(3 + extra_lookback, format='jd')).iso[0:10].replace('-', ''))
    #     vv = visits.query("day_obs > @dayobs_cutoff and (img_type == 'science' or img_type == 'acq')")
    # vv = vv.copy()
    vv = timestamp_stacker.run(vv, override=True)
    if len(vv):
        col = 'fwhm_eff'
        if col not in vv.columns:
            col = 'airmass'
        plot = bokeh.plotting.figure(y_axis_label=col, x_axis_label="Time (UTC)", height=600, width=1200)
        fig = schedview.plot.plot_visit_param_vs_time(vv, col, show_column_selector=True, hovertool=True, plot=plot)
        bokeh.io.show(fig)
else:
    print("No visits")